# Main Work Notebook

In [ ]:
# System audio I/O backend
# !sudo apt-get update && sudo apt-get install -y ffmpeg

# Python deps
!pip install silero-vad

[sudo] password for sharjeel: 
sudo: a password is required
^C
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 2.5 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [silero-vad]5 [onnxruntime]


In [ ]:
!python trim_with_silero_vad.py --in_dir ./testData/ --out_dir testWavs --keep_silence 0.25 --energy_threshold 0.4 --min_speech_ms 80 --min_silence_ms 200 --viz_dir data/viz


ok          Female_ID_41457967_11.wav
ok          Female_ID_41457967_236.wav
ok          Female_ID_41457967_268.wav
ok          Female_ID_41457967_305.wav
ok          Female_ID_41457969_1157.wav
ok          Female_ID_41457969_623.wav
ok          Female_ID_41457969_665.wav
ok          Female_ID_41457969_810.wav
ok          Female_ID_41457969_852.wav
ok          Female_ID_41457969_867.wav
ok          a-Female_ID_41457969_263.wav
